## Data Cleaning

Generation UK&I has a mission to get disadvantaged people into sustainable careers that would otherwise be unattainable. This Jupyter notebook contains the data cleaning steps taken as part of initial exploratory data analysis. EDA is captured in a separate jupyter notebook.

In [20]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas.api.types import CategoricalDtype

In [27]:
# Reading the data and combining the two data sources:
PowerBI_df = pd.read_csv("Gen_Data/data_improved_including DE_AWS.csv")
SF_df = pd.read_csv("Gen_Data/Grad_attendance_info.xls.csv")

SF_df.rename(columns={"Learner Name":"LearnerName"}, inplace=True) #renaming the column required for merging

gen_df = pd.merge(PowerBI_df, SF_df, left_on='LearnerName', right_on='LearnerName')

In [28]:
# Filter the data so that we are only looking at tech programmes:
gen_df = gen_df[gen_df["ProgramName"].isin(["Technology - Cloud Operations Engineer", 
                                                 "Technology - Data Engineering", 
                                                 "Data Analytics", "IT Support", 
                                                 "Tech Talent Accelerator - Software Engineering", 
                                                 "Google Cloud"])]

# Filter data so that we are only looking at individuals that have graduated more than 180 days ago:
gen_df = gen_df[gen_df["Days Since Cohort Graduation"] >= 180]

In [29]:
# Remove Dublin, this is the only Irish cohort and is an outlier in the dataset:
gen_df = gen_df.drop(gen_df[gen_df["Cohort_City"]=="Dublin"].index)

In [30]:
# Clean "learnerSpecialNeedsDetails" before creating a wordcloud:

Needs_clean = gen_df[["learnerSpecialNeedsDetails", "ProgramName"]].dropna()
Needs_clean = Needs_clean.drop(Needs_clean[Needs_clean["learnerSpecialNeedsDetails"]=="No"].index)
Needs_clean = Needs_clean.drop(Needs_clean[Needs_clean["learnerSpecialNeedsDetails"]=="NIL"].index)
Needs_clean = Needs_clean.drop(Needs_clean[Needs_clean["learnerSpecialNeedsDetails"]=="Non"].index)
Needs_clean = Needs_clean.drop(Needs_clean[Needs_clean["learnerSpecialNeedsDetails"]=="NONE"].index)
Needs_clean = Needs_clean.drop(Needs_clean[Needs_clean["learnerSpecialNeedsDetails"]=="nan"].index)

In [31]:
def find_text(x):
    if x != 0:
        return 1
    else:
        return 0

In [32]:
gen_df["learnerSpecialNeedsDetails_bool"] = gen_df["learnerSpecialNeedsDetails"].fillna(0)
gen_df["learnerSpecialNeedsDetails_bool"] = gen_df["learnerSpecialNeedsDetails_bool"].replace(to_replace= ["No", "NIL", "Non", "NONE", "nan"],
                                                                                              value= 0)
gen_df["learnerSpecialNeedsDetails_bool"] = gen_df["learnerSpecialNeedsDetails_bool"].apply(find_text)

In [33]:
def find_women_and_nb(x):
    if x in ["Female", "Non-binary", "Other"]:
        return 1
    else:
        return 0

In [34]:
gen_df["gender_bool"] = gen_df["gender"].apply(find_women_and_nb)

In [35]:
# replace nans in enrolment status with "Unknown"
# replace nans in employed within 90 days with 0

gen_df["Enrolment_Status"].fillna("Unknown Enrolment", inplace=True)
gen_df["Employment_Status"].fillna("Unknown Employment", inplace=True)

gen_df["Employed_within_90D"].fillna(0, inplace=True)
gen_df["Employed_within_90D_text"] = gen_df["Employed_within_90D"].replace(to_replace= [0, 1],
                                                                           value= ["Not employed 90D", "Employed 90D"])

gen_df["Employed_within_180D"].fillna(0, inplace=True)
gen_df["Employed_within_180D_text"] = gen_df["Employed_within_180D"].replace(to_replace= [0, 1],
                                                                           value= ["Not employed 180D", "Employed 180D"])

gen_df["Ethnicity_Group"].fillna("Not disclosed", inplace=True)

In [36]:
gen_df["Employment_Status"]= gen_df["Employment_Status"].replace(to_replace= ["Placed", "Unknown", "Internship"],
                                    value= ["Employed", "Unknown Employment", "Employed"])

In [37]:
gen_df["Ethnicity_Group"] = gen_df["Ethnicity_Group"].replace(to_replace= "Roma, Gypsy or Irish traveller",
                                                              value= "Other")

In [38]:
gen_df["gender"] = gen_df["gender"].replace(to_replace= ["Other/Prefer to self describe", "Choose not to identify"],
                                            value= ["Other", "Prefer not to say"])

In [39]:
gen_df["Graduation Date"] = pd.to_datetime(gen_df["Graduation Date"])
gen_df["Graduation_month"] = gen_df["Graduation Date"].dt.month
gen_df["Graduation_year"] = gen_df["Graduation Date"].dt.year
gen_df["Graduation_year_month"] = gen_df["Graduation Date"].dt.to_period("M")

In [40]:
# cleaning specific features ahead of heatmap correlation analysis:
# asylumSeeker, Has_dependents, is_ethnic_minority, is_LGBTQ+, is_lone_parent

# Cleaning asylumSeeker:
gen_df["asylumSeeker"].fillna("No", inplace=True)

# Cleaning Has_dependents:
gen_df["Has_dependents"].fillna(0, inplace=True)

# Cleaning is_ethnic_minority
gen_df["is_ethnic_minority"].fillna(0, inplace=True)

# Cleaning is_LGBTQ+
gen_df["is_LGBTQ+"].fillna(0, inplace=True)

# Cleaning is_lone_parent
gen_df["is_lone_parent"].fillna(0, inplace=True)


In [41]:
# Assigning ordinality to categorical features:

socioeconStatusMap_levels = CategoricalDtype(categories=["Exclude", "Lower socio-economic backgrounds", "Intermediate backgrounds", "Professional backgrounds"], ordered=True)
gen_df["socioeconStatusMap_ordered"] = gen_df["socioeconStatusMap"].astype(socioeconStatusMap_levels)

asylumSeeker_levels = CategoricalDtype(categories=["No", "Choose not to say", "Yes"], ordered=True)
gen_df["asylumSeeker_ordered"] = gen_df["asylumSeeker"].astype(asylumSeeker_levels)

gen_df["asylumSeeker_bool"] = gen_df["asylumSeeker"].replace(to_replace= ["No", "Choose not to say", "Yes"],
                                                                  value= [0, 0, 1])


In [42]:
gen_df.drop("LearnerName", axis=1, inplace=True)

In [44]:
gen_df.to_csv("Gen_Data/ANON_gen_data_March2024.csv")